In this notebook we are trying to forecast the Weighted price from the Bitcoin dataset(). We will use ARIMA, Auto ARIMA and Recurrent Neural Networks(RNN) methods to forecast the Bitcoin Price Data from January 2012 to August 2019(Kaggle Bitcoin DataSet).

The following articles/blogs were referred in making this notebook:
1.  https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/
2.  https://machinelearningmastery.com/time-series-data-stationary-python/
3.  https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test
4.  https://www.statisticshowto.datasciencecentral.com/unit-root/ 
5.  https://people.duke.edu/~rnau/411arim3.htm
6.  https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

In [ ]:
pip install statsmodels --upgrade

In [ ]:
!pip install  pmdarima --upgrade
import pmdarima as pm

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df_1 = pd.read_csv("../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv")

In [ ]:
df_1

In [ ]:
#Converting timestamp to datetime object
df_1.Timestamp = pd.to_datetime(df_1.Timestamp, unit='s')

# Resampling to daily frequency
df_1.index = df_1.Timestamp
df_1 = df_1.resample('D').mean()

# Resampling to weekly frequency
df_week_1 = df_1.resample('W').mean()

# Resampling to monthly frequency
df_month_1 = df_1.resample('M').mean()

In [ ]:
#Dropping the NAN values from the dataset
df = df_week_1.Weighted_Price.dropna()

In [ ]:
#Overview of the data
df

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(df[200:])
plt.xlabel("Date time")
plt.ylabel("Bitcoin Weighted Price")
plt.title(" Bitcoin Price vs Time ")
plt.show()

In [ ]:
# Create Training and Test set 
train = df[200:349]   # we are not taking into account the initial data before 2016-01  
test = df[349:]

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train,label='train')
plt.plot(test,label='test')
plt.legend()
plt.title("Train and Test set data")
plt.xlabel("Datetime")
plt.ylabel("Bitcoin weighted price")
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
#Performing the Augemented Dickey Fuller test on our data to check for stationarity in the data
print("Dickey–Fuller test: p=%f" % adfuller(train)[1])
print('Dickey-Fuller test ADF Statistic: %f' % adfuller(train)[0])

Since the p value (p=0.676423) is greater the 0.05 so we can reject the null hypothesis(The null hypothesis is that the time series is stationary.) and can say the series is not stationary.
The ADF statistic is -1.193412 which is quite negative indicating that we can strongly reject the null hypothesis. 

Our time series data is not stationary we will have to difference the series to make it stationary and determine the order of differencing that is required to make the series stationary. The order of differencing d, is inferred from the AutoCorrelation Function plots.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
# Original Series
fig, axes = plt.subplots(4, 2, sharex=True, figsize=(20,20))
axes[0, 0].plot(train.values); axes[0, 0].set_title('Original Series')
plot_acf(train.values, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(train.diff().values); axes[1, 0].set_title('1st Order Differencing')
plot_acf(train.diff().dropna().values, ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(train.diff().diff().values); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(train.diff().diff().dropna().values, ax=axes[2, 1])

# 3rd Differencing
axes[3, 0].plot(train.diff().diff().diff().values); axes[3, 0].set_title('3rd Order Differencing')
plot_acf(train.diff().diff().diff().dropna().values, ax=axes[3, 1])


plt.show()

The AutoCorrelation Function for 2nd order differencing becomes negative very quickly hinting that the series might be over differenced. So we can set the order of differencing d to 1 even though the series may not look very stationary.

The next step is to determine p which is the order of Auto Regressive(AR) terms required and q which is the order of Moving Average(MA) terms required.

The order of the AR terms p can be determind by looking at the Partial AutoCorrelation Plot (PACF) of the 1st differenced series.

In [ ]:
# PACF plot of 1st differenced series
#plt.rcParams.update({'figure.figsize':(6,3), 'figure.dpi':240})

fig, axes = plt.subplots(1, 2, sharex=True, figsize=(6,3))
axes[0].plot(train.diff().values); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
#axes[1].set(xlim=(0,300))
plot_pacf(train.diff().dropna().values, ax=axes[1])

plt.show()

The first two PACF lags are above the significance limit so we take p =2. This will also take care of the fact that the series might be slighlty under differenced.

Next we want to determine the order of q by looking at the ACF plot of the 1st differenced series. 

In [ ]:
# ACF plot of 1st differenced series


fig, axes = plt.subplots(1, 2, sharex=True, figsize=(10,5))
axes[0].plot(train.diff().values); axes[0].set_title('1st Differencing')
axes[1].set(ylim=(0,1.2))
#axes[1].set(xlim=(0,300))
plot_acf(train.diff().dropna().values, ax=axes[1])

plt.show()

The first two lags lie above the significance level. Since our series might be slightly under differenced so we set the value of q=1.

So our parameters for the ARIMA model are (1,2,1). We will now implement this model with the ARIMA package in statsmodels.

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA(train.values, order=(1,2,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

# Add comment about model paramters and P>|z|

In [ ]:
# Actual vs Fitted
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
# Forecast
fc, se, conf = model_fit.forecast(50, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

Now we will do the modeling of the time series data with Auto ARIMA

# Auto ARIMA

In [ ]:
model_auto = pm.auto_arima(train,start_p=1, start_q=1,
                      test='adf',# use to determine the d 
                      max_p=3, max_q=3,
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model_auto.summary())

The orders of the model are d=0,p=1,q=1

In [ ]:
model_auto.plot_diagnostics(figsize=(7,7))
plt.show()

# Maybe remove these diagnostics 

In [ ]:
fc, conf = model_auto.predict(n_periods=50, return_conf_int=True, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series    = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals for Auto ARIMA')
plt.legend(loc='upper left', fontsize=8)
plt.show()


Now we will look for seasonality in the data series. 

In [ ]:
!pip install scipy

In [ ]:
from scipy.signal import find_peaks

In [ ]:
#Finding peaks in the time series data
peaks, _ = find_peaks(train, height=0,distance=3)

In [ ]:
peaks

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train)
plt.plot(train[peaks], "x")
plt.show()

In [ ]:
#ignoring the first 10 peaks
peak=peaks[10:]
print(np.diff(peak).mean())

So lets take the seasonality to be 6. 

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train,label='original')
plt.plot(train.diff(6),label='seasonal differencing')
plt.legend(loc='best', fontsize=10)
plt.show()

Now we will decompose the series into the seasonality, trend and residues using the seasonal_decompose package from statsmodels. The methods used for the decomposition will be additive and multiplicative.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
result = seasonal_decompose(train, model='additive')

fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, figsize=(20,19))
#plt.title("Seasonal Decomposition")
result.observed.plot(ax=ax1).set_ylabel('Original Series')
result.trend.plot(ax=ax2).set_ylabel('Trend')
result.seasonal.plot(ax=ax3).set_ylabel('Seasonal')
result.resid.plot(ax=ax4).set_ylabel('Residue')

plt.show()

In [ ]:
result_mu = seasonal_decompose(train, model='multiplicative')

fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1, figsize=(20,19))
#plt.title("Seasonal Decomposition Multiplicative")
result_mu.observed.plot(ax=ax1).set_ylabel('Original Series')
result_mu.trend.plot(ax=ax2).set_ylabel('Trend')
result_mu.seasonal.plot(ax=ax3).set_ylabel('Seasonal')
result_mu.resid.plot(ax=ax4).set_ylabel('Residue')

plt.show()

We can see from the seasonal decomposition plot that there is annual seasonality. 

Now we will use the Auto ARIMA model with seasonality

In [ ]:
sarima_model = pm.auto_arima(train,start_p=2, start_q=2,
                      test='adf',# use to determine the d 
                      max_p=3, max_q=3,
                      m=6,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,   # No Seasonality
                      start_P=2, 
                      D=None, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(sarima_model.summary())

The Seasonal Auto Arima also finds a seasonality of 6 which matches with what we found using the peak finding technique.

In [ ]:
sarima_model.plot_diagnostics(figsize=(7,7))
plt.show()

In [ ]:
# Forecast
fc, conf = sarima_model.predict(n_periods=50, return_conf_int=True, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series    = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()


# Machine Learning Approach


Now we will extend the approch to using Recurrent Neural Networks(RNNs). 


In [ ]:
#Transformations on data and pre processing for LSTM RNNs


In [ ]:
# Sliding Window Transformation
f=4 #Size of input layer 
x= np.zeros((len(df)-f,f))
for i in range(len(df)-f):
    x[i,:]=df[i:i+f]
y=df[f:]

In [ ]:
#Normalisation
x/=df.max()

y/=df.max()

In [ ]:
#Split into train and test set
n=(350 -f) 

x_train= x[200:n,:]
x_test = x[n:,:]

y_train= y[200:n]
y_test = y[n:]

In [ ]:
plt.plot(y_train,label='train')
plt.plot(y_test,label='test')
plt.legend()
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=f))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# fit model
model.fit(x_train, y_train, epochs=len(y_train), verbose=0)
#Multilayer Perceptron model or MLP

In [ ]:
# demonstrate prediction
yhat = model.predict(x_train[-1,:].reshape((1,f)), verbose=0)
x_new = x_train[-1,:]
yhat_list = [yhat]
for i in range(y_test.size):
    _l = list(x_new[(1-f):])
    _l.append(yhat)
    x_new = np.array(_l)
    yhat = model.predict(x_new.reshape((1,f)), verbose=0)
    yhat_list.append(float(yhat))

In [ ]:
float(yhat)

In [ ]:
plt.plot(yhat_list,label='predict')
plt.plot(y_test.values,label='actual test set')
#plt.plot(y_train,label='train_data')
#plt.plot(df2,label='complete')
plt.legend()
plt.show()

Comparing the output from the RNN model to Seasonal Auto Arima

In [ ]:
# Plot

fc_ml_series    = pd.Series(yhat_list, index=test.index)
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train/df.max(), label='training')
plt.plot(test/df.max(), label='actual')
plt.plot(fc_series/df.max(), label='forecast ARIMA')
plt.plot(fc_ml_series, label='forecast RNN')
plt.fill_between(lower_series.index, lower_series/df.max(), upper_series/df.max(), 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals for ARIMA AND RNN')
plt.legend(loc='upper left', fontsize=8)
plt.show()
